# Business Case Study - Staff Planning

# Problem Statement

 The insurance approval process, i.e., underwriting is one of the important and time-consuming tasks in an insurance application processing. When you submit your insurance application, the underwriter of the company evaluates it based on the details that you provide using a rule-based or an ML model and decides whether or not to approve your application. An insurance company InsurePlus wants you to help them with finding the optimal number of staff that they need for their insurance application approval process for the calendar year 2021. In the industry, the number of staffs is considered as a continuous variable. This is also called a staff Full-Time Equivalent. For example, if a full-time employee (FTE =1) works for 50 hours a week, 10 hours corresponds to 0.2 FTEs.

- The company operates in three states A, B, and C.
- The company can either handle an application with the staff that they hire or outsource it to a vendor. (Assume that there is no capacity limitation to outsourcing.)
- If they hire staff, he/she can handle 40 insurance applications per month when he/she works 100% of the workdays. However, there are days that he/she will be unavailable to process applications due to training, off days, etc.
- A staff member’s availability (in percentage) to work on processing the insurance applications for each month is shown in the table given below. As mentioned before, with 100% availability, each member can handle 40 applications.

A special note of practical relevance: In the industry, staff availability is predicted using a time-motion study. But in this case, you have been given fixed numbers for each month. States A and B have a regulatory restriction that the outsourced insurance applications cannot be more than 30% and 40% of the total number of applications for each month, respectively. The objective is to optimise the total cost for the application approval process by distributing the right number of applications between the FTEs and the vendors while meeting the monthly demand for each state at the same time.



## Importing the libraries and reading the dataset

In [1]:
# Importing Libraries 

import pandas as pd
import numpy as np
import math
from pyomo.environ import *

In [2]:
from __future__ import division
from pyomo.opt import SolverFactory

In [3]:
# Reading the data from Excel workbook - Staffing.xlsx

InputData = 'Staffing+Data.xlsx'

#Read the data from Demand Data sheet
Demand = pd.read_excel(InputData,sheet_name='DemandData')

#Read the data from Staff Availability sheet
Availability = pd.read_excel (InputData,sheet_name='StaffAvailability')

#Read the data from Cost sheet
Cost = pd.read_excel (InputData,sheet_name='Cost')

#Max Applications Handled by FTE per month
transcost = pd.read_excel( InputData, sheet_name ='ServiceRate')

In [4]:
#Printing the dataframe Demand

Demand.head()

,State,Month,Demand
0,A,Jan,5240
1,A,Feb,4878
2,A,Mar,5942
3,A,Apr,2297
4,A,May,1992


In [5]:
#Printing the dataframe Availability

Availability.head()

,State,Month,LB,UB,StaffAvPer
0,A,Jan,0.70,0.90,0.81
1,A,Feb,0.65,0.85,0.76
2,A,Mar,0.70,0.80,0.75
3,A,Apr,0.75,0.85,0.80
4,A,May,0.70,0.85,0.78


In [6]:
#Printing the dataframe Cost

Cost.head()

,State,Month,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,60000,5000.0,180
1,A,Feb,60000,5000.0,180
2,A,Mar,60000,5000.0,180
3,A,Apr,60000,5000.0,180
4,A,May,60000,5000.0,180


In [7]:
# Consolidating the three dataframes into single dataframe for further analysis

data=pd.merge(Demand,Availability ,on=['State','Month'])
Staff=pd.merge(data,Cost, on = ['State','Month'])
Staff.head()

,State,Month,Demand,LB,UB,StaffAvPer,AnnualSalary,MonthlySalary,UnitOutSourceCost
0,A,Jan,5240,0.70,0.90,0.81,60000,5000.0,180
1,A,Feb,4878,0.65,0.85,0.76,60000,5000.0,180
2,A,Mar,5942,0.70,0.80,0.75,60000,5000.0,180
3,A,Apr,2297,0.75,0.85,0.80,60000,5000.0,180
4,A,May,1992,0.70,0.85,0.78,60000,5000.0,180


In [8]:
Staff.shape

(36, 9)

In [10]:
Staff.isnull().sum()

State                0
Month                0
Demand               0
LB                   0
UB                   0
StaffAvPer           0
AnnualSalary         0
MonthlySalary        0
UnitOutSourceCost    0
dtype: int64

# Question 1

The company wants to know the optimised staffing recommendations for the business case described. 
Write the mathematical model for the deterministic optimisation problem. Define and explain your decision variables, objective function and the constraint. (Hint: Use months of the year as the model timeline).



##  Mathematical formulation / Pyomo components

---

**Sets:** 

The indexes for the given problem are, <br>
- State, $s \in State$
- Month, $m \in Month$

---

**Parameters:** 

- $\text{app_FTE}_{s,m} \text{ - Applications processed by FTE}$ <br>


- $\text{app_outsourced}_{s,m} \text{ - Applications outsourced}$ <br>


- $\text{msalary_FTE}_{s,m} \text{ - Monthly Salary of FTE}$ <br>


- $\text{unitcost_outsourced}_{s,m} \text{ - Unit Cost for outsourcing 1 application}$ <br>


- $\text{staff_avail}_{s,m} \text{ - Average availability of FTE}$ <br>


- $\text{lb}_{s,m} \text{ - Minimum availability of FTE}$ <br>


- $\text{ub}_{s,m} \text{ - Maximum availability of FTE}$ <br>


- $\text{max_app_served} \text{ - Number of insurance applications that can be processed by an FTE in a month when working with 100 percent availability}$ <br>




- $\text{outsourced_restA} \text{ - Maximum Number of insurance applications that can be outsourced in a month for State A}$ <br>


- $\text{outsourced_restB} \text{ - Maximum Number of insurance applications that can be outsourced in a month for State B}$ <br>
                                                                                                             
                                                                                                             
---


**Decision variables:**
                                                                                                             
- Total number of  applications processed by FTE for each state and month combination <br>

 $\text{app_FTE}_{s,m} \,\,\,\,\,\,\, \text{where} \ s \in State, m \in Month $<br>
 
 
- Total number of  applications outsourced for each state and month combination <br>

 $\text{app_outsourced}_{s,m} \,\,\,\,\,\,\, \text{where} \ s \in State, m \in Month $<br>


---

**Objective Function:** <br>

To minimize the total cost of insurance approval process by distributing the right number of applications between FTE and outsourced.



\begin{align}
\textrm{min} (\sum \limits _{s,m} \text{app_FTE}_{s,m}* \text{msalary_FTE}_{s,m} + \sum \limits _{s,m} \text{app_outsourced}_{s,m}* \text{unitcost_outsourced}_{s,m})
\end{align}
where $s \in State$ and $m \in Month$

---

**Constraints:**


- **Demand Constraint** :- Demand of Insurance for each state and month combination


>- $\sum \limits _{s,m} \text{app_FTE}_{s,m}* \text{staff_avail}_{s,m}*\text{max_app_served} == \text{demand} \ \ \ \ \forall  {s \in State, m \in Month}$


- **Staff Availability** :- Staff Availability for best and worst case scenerios.


>- $ \text{staff_avail}_{s,m}  > \text{lb}_{s,m} \ \ \ \ \forall  {s \in State, m \in Month}$

>- $ \text{staff_avail}_{s,m}  < \text{ub}_{s,m} \ \ \ \ \forall  {s \in State, m \in Month}$



- **Outsourcing Regulations** :- Regulatory Regulations on States A & B for outsourcing applications.

>- $ \text{app_outsourced}_{s,m}  < (\text{outsourced_restA}_{s,m}*\text{app_outsourced}_{s,m}) \,\,\,\,\,\,\, \text{where} \ s \in A , m \in Month $

>- $ \text{app_outsourced}_{s,m}  < (\text{outsourced_restB}_{s,m}*\text{app_outsourced}_{s,m}) \,\,\,\,\,\,\, \text{where} \ s \in B , m \in Month $


# Question 2

Code the problem is Python and use any optimization package to solve it. Add comments to your code to explain each step. 

## Data pre-processing 

In [12]:
# Create the required Python data structures for indexes and parameters

# Create a Dictionary object Demand with index: 'State' and 'Month' and Value as "Demand" column from the Staff Dataframe

Demand = Staff.set_index(['State','Month'])['Demand'].to_dict()

# Create a Dictionary object FTE_Salary with index: 'State' and 'Month' and Value as "MonthlySalary" column from the Staff Dataframe

FTE_Salary = Staff.set_index(['State','Month'])['MonthlySalary'].to_dict()

# Create a Dictionary object UnitOutSourceCost with index: 'State' and 'Month' and Value as "UnitOutSourceCost" column from the Staff Dataframe

UnitOutSourceCost = Staff.set_index(['State','Month'])['UnitOutSourceCost'].to_dict()

FTE_AppServRate = 40

StaffAvPer = Staff.set_index(['State','Month'])['StaffAvPer'].to_dict()
StaffAv_LB = Staff.set_index(['State','Month'])['LB'].to_dict()
StaffAv_UB = Staff.set_index(['State','Month'])['UB'].to_dict()

State = Staff['State'].unique()
Month = Staff['Month'].unique()


#### Expected output:

Create a data frame containing the number of outsourced applications  and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. 

In [13]:
# Creating a model instance

model = ConcreteModel()

In [14]:
# Define Pyomo sets and Parameters


model.i = Set(initialize=State.tolist(),doc='States')
model.j = Set(initialize=Month.tolist(),doc='Months')
model.demand = Param(model.i,model.j,initialize = Demand,doc='Demand')
model.sa = Param(model.i,model.j,initialize = StaffAvPer,doc = 'StaffAvPercent')

In [15]:
# Decision variables

model.x = Var(model.i,model.j,doc='No of FTE',domain = NonNegativeReals)
model.y = Var(model.i,model.j,doc='No of Outsource App',domain = NonNegativeIntegers)

In [16]:
# Constraints

# Demand Constraint
def total_demand(m,i,j):
    return (m.x[i,j]*40*m.sa[i,j] + m.y[i,j] == m.demand[i,j])
            
model.total_demand = Constraint(model.i,model.j, rule=total_demand)

In [17]:
# Outsource Demand Constraints

model.outsource_demand = ConstraintList()
for j in model.j:
    for i in model.i:
        if i == 'A':
            model.outsource_demand.add(expr = model.y[i,j] <= 0.3*model.demand[i,j])
        elif i == 'B':
            model.outsource_demand.add(expr = model.y[i,j] <= 0.4*model.demand[i,j])

In [18]:
# Objective function

model.value = Objective(expr = sum(sum(model.x[i,j]*FTE_Salary[i,j] + model.y[i,j]*UnitOutSourceCost[i,j] for i in model.i) for j in model.j), sense= minimize)

In [19]:
# Invoking the solver

result = SolverFactory('glpk').solve(model)
result.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 17962336.4487699
  Upper bound: 17962336.4487699
  Number of objectives: 1
  Number of constraints: 61
  Number of variables: 73
  Number of nonzeros: 97
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.08120131492614746
# ----------------------------------------------------------
#   Solution Information
# ------------

In [20]:
# Print the value of the objective function

optimized_cost = round(model.value()/1000000,2)
optimized_cost

17.96

**`Checkpoint 1:`** Seems like the company has to spend around 17.9 m$ in total for the application approval process.

In [21]:
model.pprint()

8 Set Declarations
    demand_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :   36 : {('A', 'Jan'), ('A', 'Feb'), ('A', 'Mar'), ('A', 'Apr'), ('A', 'May'), ('A', 'Jun'), ('A', 'Jul'), ('A', 'Aug'), ('A', 'Sep'), ('A', 'Oct'), ('A', 'Nov'), ('A', 'Dec'), ('B', 'Jan'), ('B', 'Feb'), ('B', 'Mar'), ('B', 'Apr'), ('B', 'May'), ('B', 'Jun'), ('B', 'Jul'), ('B', 'Aug'), ('B', 'Sep'), ('B', 'Oct'), ('B', 'Nov'), ('B', 'Dec'), ('C', 'Jan'), ('C', 'Feb'), ('C', 'Mar'), ('C', 'Apr'), ('C', 'May'), ('C', 'Jun'), ('C', 'Jul'), ('C', 'Aug'), ('C', 'Sep'), ('C', 'Oct'), ('C', 'Nov'), ('C', 'Dec')}
    i : States
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}
    j : Months
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {'Jan', 'Feb', 'Mar', 'Apr',

In [22]:
# Creating dataframe for the results

Output_actual = []

for i in model.i:
    for j in model.j:
        
        #cost for the application approval process
        no_of_fte=model.x[i,j].value
        demand = model.demand[i, j]
        no_of_outsource_app = model.y[i,j].value
        no_of_FTE_processed_app = model.x[i,j].value*40*model.sa[i,j]
        cost = model.x[i,j].value*FTE_Salary[i,j] + model.y[i,j].value*UnitOutSourceCost[i,j]
        percent_outsourced_app = round((no_of_outsource_app/demand)*100,2)
        avg_cost_per_app = round(cost/demand,2)
        
        Output_actual.append([i, j, no_of_fte,demand, no_of_outsource_app,no_of_FTE_processed_app, cost, percent_outsourced_app,avg_cost_per_app])
        
print(Output_actual)

[['A', 'Jan', 161.728395061728, 5240, 0.0, 5239.999999999987, 808641.97530864, 0.0, 154.32], ['A', 'Feb', 160.460526315789, 4878, 0.0, 4877.999999999985, 802302.631578945, 0.0, 164.47], ['A', 'Mar', 198.066666666667, 5942, 0.0, 5942.000000000011, 990333.333333335, 0.0, 166.67], ['A', 'Apr', 71.78125, 2297, 0.0, 2297.0, 358906.25, 0.0, 156.25], ['A', 'May', 63.8461538461538, 1992, 0.0, 1991.9999999999986, 319230.76923076896, 0.0, 160.26], ['A', 'Jun', 77.9109589041096, 2275, 0.0, 2275.0000000000005, 389554.794520548, 0.0, 171.23], ['A', 'Jul', 137.279411764706, 5334, 1600.0, 3734.000000000003, 974397.05882353, 30.0, 182.68], ['A', 'Aug', 110.888157894737, 3371, 0.0, 3371.0000000000055, 554440.7894736851, 0.0, 164.47], ['A', 'Sep', 116.018518518519, 3759, 0.0, 3759.000000000016, 580092.592592595, 0.0, 154.32], ['A', 'Oct', 120.856164383562, 3529, 0.0, 3529.0000000000105, 604280.82191781, 0.0, 171.23], ['A', 'Nov', 110.257352941176, 4284, 1285.0, 2998.9999999999873, 782586.76470588, 30.0,

In [24]:
Output_actual = pd.DataFrame(Output_actual, columns = ['State', 'Month', 'No of FTE',"Demand",'No of outsource app',"No of FTE processed App" , 'Estimated Cost', 'percent outsourced app', 'average cost per application'])

Output_actual[['Estimated Cost in mn $']] = Output_actual[['Estimated Cost']]/1000000
Output_actual['Demand'] = Output_actual['Demand'].astype(int)
Output_actual['No of outsource app'] = Output_actual['No of outsource app'].astype(int)
Output_actual['No of FTE processed App'] = Output_actual['No of FTE processed App'].astype(int)
Output_actual = Output_actual.round({"No of FTE":1, "No of outsource app":0, "Estimated Cost in mn $":1})

In [25]:
Output_actual

,State,Month,No of FTE,Demand,No of outsource app,No of FTE processed App,Estimated Cost,percent outsourced app,average cost per application,Estimated Cost in mn $
0,A,Jan,161.7,5240,0,5239,808641.975309,0.00,154.32,0.8
1,A,Feb,160.5,4878,0,4877,802302.631579,0.00,164.47,0.8
2,A,Mar,198.1,5942,0,5942,990333.333333,0.00,166.67,1.0
3,A,Apr,71.8,2297,0,2297,358906.250000,0.00,156.25,0.4
4,A,May,63.8,1992,0,1991,319230.769231,0.00,160.26,0.3
5,A,Jun,77.9,2275,0,2275,389554.794521,0.00,171.23,0.4
6,A,Jul,137.3,5334,1600,3734,974397.058824,30.00,182.68,1.0
7,A,Aug,110.9,3371,0,3371,554440.789474,0.00,164.47,0.6
8,A,Sep,116.0,3759,0,3759,580092.592593,0.00,154.32,0.6
9,A,Oct,120.9,3529,0,3529,604280.821918,0.00,171.23,0.6


In [26]:
# Writing the results in to an Excel sheet
from openpyxl import load_workbook
book = load_workbook("Staffing+Data_Result.xlsx")

# create excel writer object
writer = pd.ExcelWriter("Staffing+Data_Result.xlsx", engine = 'openpyxl')


#Assigning the workbook to the writer object
writer.book = book


# write dataframe to excel sheet named 'output_actual'
Output_actual.to_excel(writer, sheet_name='Output_actual_Q2')


# save the excel file
writer.save()
print('Output_actual_Q2 data is written successfully to Excel Sheet.')

Output_actual_Q2 data is written successfully to Excel Sheet.


# Question 3

#### Worst-case and best-case analysis based on the staffs' availability.

Assuming that the distribution is the same across all the states,

#### 3.1 Worst case analysis 

- 3.1.1 What is the optimal number of staff members for the worst case? 

- 3.1.2 What is the percentage of outsourcing for the worst case? 

- 3.1.3 What is the average cost per application for the worst case?


#### 3.2 Best case analysis 

- 3.2.1 What is the optimal number of staff members for the best case? 

- 3.2.2 What is the percentage of outsourcing for the best case? 

- 3.2.3 What is the average cost per application for the best case?


#### Expected output:

For each of the subtasks (3.1 and 3.2) create a data frame containing the number of outsourced applications and the number of FTEs for each state-month combination. You can choose to have extra columns like staff availability, demand etc. in your dataframe apart from the ones mentioned earlier. Also, print the overall average percentage of outsourced applications and the overall average cost per application. 

### 3.1 Worst Case Analysis 

#### 3.1.1 Optimal number of staff members

In [ ]:
# Creating a model instance


In [ ]:
# Define Pyomo sets and Parameters


In [ ]:
# Decision variables


In [ ]:
# Constraints


In [ ]:
# Objective function


In [ ]:
# Invoking the solver


In [ ]:
# Print the value of the objective function


**`Checkpoint 2:`** The company has to spend around 19.6 m$ in total for the application approval process if the staffs are working with the minimum availability.

In [ ]:
# Creating dataframe for the results


#### 3.1.2 Percentage of outsourced applications 

In [ ]:
# write your code here


#### 3.1.3 Average cost per application

In [ ]:
# write your code here


### 3.2  Best Case Analysis 

#### 3.2.1 Optimal number of staff members


In [ ]:
# Creating a model instance


In [ ]:
# Define Pyomo sets and Parameters


In [ ]:
# Decision variables


In [ ]:
# Constraints


In [ ]:
# Objective function


In [ ]:
# Invoking the solver


In [ ]:
# Print the value of the objective function


**`Checkpoint 3:`** The company has to spend around 16.5 m$ in total for the application approval process if the staffs are working with the maximum availability.

In [ ]:
# Creating dataframe for the results


#### 3.2.2 Percentage of outsourced applications

In [ ]:
# write your code here


#### 3.2.3 Average cost per application

In [ ]:
# write your code here


# Question 4

#### Creating Visualisations

Create the following visualisations using your preferred method (i.e. Python, PowerPoint, Power BI, etc.) and add it to your report. 

Use the solution of Q2 to create a stacked column chart that shows the percentage of applications processed by the staff and by the vendor for each month (%staff processed applications+ %vendor processed applications should add up to 100%). 
Create a graph to show how the cost per application increases with respect to any change in the parameters in your analysis.
Hint: Use the cost per application that you calculate in Questions 2 and 3 (i.e., the best case, and the worst case). 

**Note:** You can create the charts in Python or some other visualisation tools and make it a part of your final report directly.